# Towards a definition of the simulation framework for a dimer system

In [203]:
import os
import sys
import copy as cp
import numpy as np
import string as st
import time as tm
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as anm

from IPython.display import display, HTML
%matplotlib inline

## Simulation Parameters

The first thing we need is to define a parameters object. 
The parameters needed for the simulation are: 
* sim_name: simulation name. This is then used for the script, input and trayectory files
* dir_name: this is the place where all the lammps files are placed
* space: this is the input to the "region" comand. space should be a dictionary, where we give the region specification for x y and z, and the type of boundaries (where for this system is either periodic or stretch).
    * ¿How should I define the region? it could be a list of either 3 or 6 values. 
        * If 3 then [w l h]. If 6 [xmin xmax ymin ymax zmin zmax]
    * boundary vector of strings
* Temperature
* pair_style cutoff
* number of atoms (maybe overriden by input script)
* Particle Properties
    * initial position
    * radius
    * susceptibility
    * drag
* Field Parameters:
    * Magnitude
    * Frequency
    * Angle
* Run parameters
    * Timestep
    * framerate
    * total time

In [93]:
class particle_properties():
    def __init__(self,position,**kargs):
        """
        Initializes a set of particle properties. Normally, for each particle we define a particle properties object. 
        the required parameter is the initial position
        the position has to be either the string "random" or a list of three numbers. 
        It the position is specified as "random", the program requires also a space specified. 
        
        Other optional parameters are:
            radius (default = 4um)
            susceptibility (default = 1)
            diffusion (default = 1um^2/s)
        """
        # defaults
        self.radius = 4 #um
        self.susceptibility = 1
        self.diffusion = 1 #um^2/s
        
        if (not isinstance(position, (str))) & (len(position)==3):
            self.initial_position = [position[0],position[1],position[2]] #um
        elif position.lower() == "random".lower():
            if 'space' in kargs:
                space = kargs['space']
                if len(space['region'])==3:
                    self.initial_position = [s*r-s/2 for s,r in zip(space['region'],np.random.rand(3))]
                if len(space['region'])==6:
                    size = space['region'][1::2]-space['region'][0::2]
                    center = space['region'][1::2]+space['region'][0::2]/2
                    self.initial_position = [s*r-s/2 for s,r in zip(size,np.random.rand(3))]
                    self.initial_position = [r+c for s,r in zip(center,self.initial_position)]
            else:
                #If we expect a random starting position, we need Space to be defined. 
                #that can be input either by introducing the sim_parameters or the space dictionary.
                raise Exception("I can't place particles randomly if I don't know the coordinates of the space")
        else:
            raise Exception("This is an invalid value for the required argument position")
        
        if 'susceptibility' in kargs: self.susceptibility = kargs['susceptibility']
        if 'diffusion' in kargs: self.radius = kargs['diffusion']
        
        if 'radius' in kargs:
            self.radius = kargs['radius']
            if 'diameter' in kargs:
                warn('You have too many particle size specifications')
        elif 'diameter' in kargs:
            self.radius = kargs['diameter']/2
            if 'volume' in kargs:
                warn('You have too many particle size specifications')
        

This are really more suitable as internal parameters to the lammps script. The mass of the object determines only its relation between the damping and the diffusion coefficient.There is no effect of gravity. If it is implemented, it would be better specified by a relative density parameter

    damp = 0.03e-6 #s
    temperature = float(300) #K 
    kb = float(4/300) #pN nm
    self.mass = kb*temperature*damp/self.diffusion

In [187]:
class run_parameters():
    def __init__(self,**kargs):
        """ 
        Optional keyword parameters are:
        timestep (default = 1e-3 sec)
        framerate (default = 30 sec)
        total_time (default = 60 sec)
        """
        
        self.timestep = 1e-3 #s
        self.framerate = 30 #s
        self.total_time = 60 #s
        
        if 'timestep' in kargs: self.timestep = kargs['timestep']
        if 'framerate' in kargs: self.frameraate = kargs['franerate']
        if 'total_time' in kargs: self.total_time
        
class field_parameters():
    def __init__(self,**kargs):
        """
        Optional keyword parameters are:
        magnitude (= 10mT)
        frequency (= 10Hz)
        angle (= 30º)
        dipole_cutoff (= 200um)
        lj_cutoff (=10um)
        walls (=[-5um,5um])
        """
        self.magnitude = 10 #mT
        self.frequency = 10 #Hz
        self.angle = 30 #degrees
        self.dipole_cutoff = 200 #um
        self.lj_cutoff = 10 #um
        self.walls = [-5,5]
        
        if 'magnitude' in kargs: self.magnitude = kargs['magnitude']
        if 'frequency' in kargs: self.frequency = kargs['frequency']
        if 'angle' in kargs: self.angle = kargs['angle']
        if 'dipole_cutoff' in kargs: self.dipole_cutoff = kargs['dipole_cutoff']
        if 'lj_cutoff' in kargs: self.lj_cutoff = kargs['lj_cutoff']
        if 'walls' in kargs: self.walls = kargs['walls']
        
class sim_parameters():
    def __init__(self,**kargs):
        """
        Optional keyword parameters are:
        temperature (=300K)
        space (={"region":[200,200,20],"boundary":["s","s","f"]})
        file_name (= test)
        dir (="")
        stamp_time (=False) this parameter determines if the file_name is modified by a timestamp.
            This is important to prevent overwriting experimetns
        """
        self.temperature = 300
        self.space = {"region":[200,200,20],"boundary":["s","s","f"]}
        self.file_name = "test"
        self.dir = ""
        self.stamp_time = False
        
        if 'temperature' in kargs: self.temperature = kargs['temperature']
        if 'space' in kargs: self.space = kargs['space']
        if 'file_name' in kargs: self.file_name = kargs['file_name']
        if 'dir' in kargs: self.dir = kargs['dir']
        if 'stamp_time' in kargs: self.stamp_time = kargs['stamp_time']
            
        if len(self.space["region"])==3:
            self.space["region"] = [p*s/2 for s in self.space["region"] for p in [-1,1]]

## Defining a Simulation

We group all the previous parameters in a lmp_sim class, which contains methods to setup, run and read the simulation. 

In [237]:
class lmp_sim():
    def __init__(self,*pargs,**kargs):
        """
        positions should be a numpy array with three columns. The three columns are x, y and z. 
        
        By default, all particles will have the same particle_properties, which are: 
            radius (default = 4um)
            susceptibility (default = 1)
            diffusion (default = 1um^2/s)
        
        however, particles can have different particle parameters by specifying the keyword argument:
            particle_properties
        which must be either an array of particle_parameters objects, or a single particle_parameters object.
        
        If both, particle_properties and positions are specified, then the initial positions of 
        particle_properties are overwritten. 
            
        All of the other keyword arguments are passed to the parameters classes. These are:
        
        sim_parameters
            temperature (=300K)
            space (={"region":[200,200,20],"boundary":["s","s","f"]})
            file_name (= test)
            dir (="")
            stamp_time (=False) this parameter determines if the file_name is modified by a timestamp.
                This is important to prevent overwriting experiments
        field_parameters
            magnitude (= 10mT)
            frequency (= 10Hz)
            angle (= 30º)
            dipole_cutoff (= 200um)
            lj_cutoff (=10um) 
        run_parameters
            timestep (default = 1e-3 sec)
            framerate (default = 30 sec)
            total_time (default = 60 sec)
        """
        if len(pargs)>=1:
            self.positions = pargs[0]
        else:
            self.positions = []
        
        if 'sim_parameters' in kargs: self.sim_parameters = karg['sim_parameters']
        else: self.sim_parameters = sim_parameters()
        if 'field_parameters' in kargs: self.field_parameters = kargs['field_parameters']
        else: self.field_parameters = field_parameters()
        if 'run_parameters' in kargs: self.run_parameters = kargs['run_parameters']
        else: self.run_parameters = run_parameters()
            
        if 'particle_properties' in kargs: 
            if len(kargs['particle_properties'])==1:
                # if a single particle property is sent, every particle must have this parameter. 
                # for this I need the positions vector not to be empty
                # if the positions vector is empty, the particle_properties value sets the position of a single particle
                if self.positions:
                    self.particle_properties = [kargs['particle_properties'] for p in self.positions]
                    for i,prop in enumerate(self.particle_properties):
                        prop.initial_position = self.positions[i]
                else:
                    self.particle_properties = [kargs['particle_properties']]
                    self.positions = np.array([self.particle_properties[0].initial_position])
                        
            elif kargs['particle_properties']:
                if self.positions & (len(self.positions)==len(kargs['particle_properties'])):
                    # if an array of particle_properties is passed,
                    # the position vector is not empty,
                    # and if the positions vector and the array have the same size, then 
                    # the positions vector overrides the initial positions in the properties array. 
                    self.particle_properties = kargs['particle_properties']
                    for i,prop in enumerate(self.particle_properties):
                        prop.initial_position = self.positions[i]
                elif not self.positions:
                    # if an array of particle_properties is passed,
                    # but the positions vector is not, then
                    # the positions vector is defined by the initial positions in the properties array. 
                    self.particle_properties = kargs['particle_properties']
                    self.positions = np.array([prop.initial_position for prop in self.particle_properties])
                else:
                    # the only remaining option is that particles positions were passed, and property arrays, 
                    # but of different size. This is an error.
                    raise Exception("the array of particle_properties and positions must have the same length")
        else:
            # if particle properties was not passed, then simply create default particle properties 
            # with the positions vector. 
            # If the position vector was not passed, raise an error. 
            if self.positions:
                self.particle_properties = [particle_properties(p) for p in self.positions]
            else:
                raise Exception("You must pass either a position vector or a particle_properties array.")

    def generate_scripts(self,**kargs):
        """
        This method generates the input script for the lammps simulation. 
        It accepts some options, which for now include:
        * input_file: boolean, (False) which specifies if the input file is stored separatelly from the main script. 
            This icreases readibility for large amounts of particles
        """
        if 'input_file' in kargs: self.input_file = kargs['input_file']
        else: self.input_file = False
            
        if self.input_file:
            raise Exception("Sorry. This will be a feature in the future, but it is not programmed yet")
        
        
        self.base_name = os.path.join(self.sim_parameters.dir,self.sim_parameters.file_name)
        if self.sim_parameters.stamp_time:
            self.base_name = self.base_name + \
                tm.strftime('_%Y_%m_%d_%H_%M_%S')

                
        self.script_name = self.base_name+'.lmpin'
        self.output_name =  self.base_name+'.lammpstrj'
        
        preamble = st.Template(
            "## ---Preamble--- ## \n" +
            "units micro \n" +
            "atom_style hybrid sphere paramagnet \n" +
            "boundary $x_bound $y_bound $y_bound\n" +
            "neighbor 4.0 nsq \n" +
            "pair_style lj/cut/dipole/cut $lj_cut $dpl_cut \n")
        
        preamble = preamble.substitute(x_bound = self.sim_parameters.space["boundary"][0],
                           y_bound = self.sim_parameters.space["boundary"][1],
                           z_bound = self.sim_parameters.space["boundary"][2],
                           lj_cut = self.field_parameters.lj_cutoff,
                           dpl_cut = self.field_parameters.dipole_cutoff)
        
        atoms = "\n".join(
                [st.Template("create_atoms 1 single $x0 $y0 $z0").substitute(x0=p[0],y0=p[1],z0=p[2])
                 for p in self.positions])+"\n\n"
            
        world = st.Template(
            "\n## ---Set World-- ## \n" +
            "region space block $spx1 $spx2 $spy1 $spy2 $spz1 $spz2 # this is in microns \n" +
            "create_box 1 space \n\n"+
            atoms + 
            "group Atoms type 1 \n" + 
            "pair_coeff * * 1.0 1.0 \n")
        world = world.substitute(spx1 = self.sim_parameters.space["region"][0],
                        spx2 = self.sim_parameters.space["region"][1],
                        spy1 = self.sim_parameters.space["region"][2],
                        spy2 = self.sim_parameters.space["region"][3],
                        spz1 = self.sim_parameters.space["region"][4],
                        spz2 = self.sim_parameters.space["region"][5])
            
        particle_props = st.Template(
            "\n## ---Particle Properties---## \n" +
            "mass * 1 \n" +
            "set group all susceptibility 1 \n" +
            "\n")
        particle_props = particle_props.substitute()
         
        field = st.Template(
            "## ---Fixes---## \n" + 
            "variable Bmag atom $Bmag \n" + 
            "variable freq atom $freq \n" + 
            "variable theta atom $angle \n" + 
            "variable fieldx atom v_Bmag*sin(v_freq*time)*cos(v_theta) \n" + 
            "variable fieldy atom v_Bmag*sin(v_freq*time)*cos(v_theta) \n" + 
            "variable fieldz atom v_Bmag*sin(v_freq*time)*sin(v_theta) \n\n")
        
        field = field.substitute(Bmag = self.field_parameters.magnitude,
                        freq = self.field_parameters.frequency,
                        angle = self.field_parameters.angle)

        fixes = st.Template(
            "fix 	1 Atoms bd 300.000000 0.030000 1 \n" + 
            "fix 	2 Atoms setdipole v_fieldx v_fieldy v_fieldz \n")
        fixes = fixes.substitute()
            
        run = st.Template(
            "\n## ---Run Commands--##\n"
            "timestep 	1000 \n" + 
            "dump 	3 all custom 100 $out_name id type x y z mu \n" + 
            "thermo_style 	custom step atoms \n" + 
            "thermo 	100 \n" + 
            "run 	60000 \n")
        run = run.substitute(out_name = self.output_name)
        
        f = open(self.script_name,'w')
        f.write(preamble)
        f.write(world)
        f.write(particle_props)
        f.write(field)
        f.write(fixes)
        f.write(run)
        f.close
        
    def run_sim(self):
        """This function runs an input script named filename in lammps. The input should be located in target_dir"""
        if sys.platform=='darwin':
            lmp_exec = "./lmp_mac"
        else:
            lmp_exec = "lmp_mingw64.exe"

        os.system(lmp_exec + " -in "+self.script_name)

        self.output_name = self.base_name

In [238]:
sim1 = lmp_sim([[-10,0,0],[10,0,0]])
sim1.generate_scripts()

sim1.run_sim()

This is getting too large. I think it's time to transfer it to an outside function

### Timestamp behaviour. 

It seems I have some options regarding the behaviour of the lmp_sim object regarding several simulations. 
* One option is to assign a timestamp to a single lmp_sim object. 
    * This means that for successive simulations I would create a new lmp_sim object. 
    * But it is not clear what would happen if, for example, I modify the parameters. Would this create a new script and new data file, but leave the old one thrown away? or maybe rewrite everything even if I loose the data? This last option seems cleaner, like a hard link between a simulation and a timestamp, so that if I want to see further simulations I need to copy the object. But it risks loosing data. 
    * In this case I also have to think what the timestamp represents. One option is the script creation time. The other is the run time. If it were the run time, successive runs would change the timestamp, and I would have to explicitly erase the previous file to keep the timestamp-lmp_sim link.
    * what happens to the lmp_sim-timestamp link when I quit the kernel? the lmp_sim object is erased, so the timestamp objects should be erased unless I specifically save them? I could then make a method to read a script file into a lmp_sim object. 
* The other option is to assign a timestamp when the script is generated, in such a way that I can run several simulations from the same object. This seems economical in terms of memory, but the simulation reading might be cumbersome because:
    * When I run a new simulation I assign a new timestamp to the lmp_sim object. If I want to read the previous simulation I have to either keep a list of timestamps generated by a lmp_sim object, or something